#Team-7: Mini Project
##Team members:

Dharshni.S (2020PITAIDS103) -211520243012

Mamatha.J (2020PITAIDS109) - 211520243031

##Team Guide:

Dr.T. Kalaichelvi , M.E., Ph.D., HOD, AI&DS.


In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [1]:
# setting random seed for notebook reproducability
rnd_seed=23
np.random.seed=rnd_seed
np.random.set_state=rnd_seed

NameError: ignored

## 2. Creating the Spark Session

In [ ]:
spark = SparkSession.builder.master("local[2]").appName("Linear-Regression-California-Housing").getOrCreate()

In [ ]:
spark

In [ ]:
sc = spark.sparkContext
sc

<SparkContext master=local[2] appName=Linear-Regression-California-Housing>

In [ ]:
sqlContext = SQLContext(spark.sparkContext)
sqlContext

## 3. Load The Data From a File Into a Dataframe

In [ ]:
TOUR_DATA= 'Country Quater Wise Visitors.csv'

In [ ]:
tour_df = spark.read.csv(path=TOUR_DATA,header=True,inferSchema=True)

Specifying the schema when loading data into a DataFrame will give better performance than schema inference.

In [ ]:
# define the schema, corresponding to a line in the csv data file.
schema = StructType([
    StructField("_c0", FloatType(), nullable=True),
    StructField("_c1", FloatType(), nullable=True),
    StructField("_c2", FloatType(), nullable=True),
    StructField("_c3", FloatType(), nullable=True),
    StructField("_c4", FloatType(), nullable=True),
    StructField("_c5", FloatType(), nullable=True),
    StructField("_c6", FloatType(), nullable=True),
    StructField("_c7", FloatType(), nullable=True),
    StructField("_c8", FloatType(), nullable=True),
    StructField("_c9", FloatType(), nullable=True),
    StructField("_c10", FloatType(), nullable=True),
    StructField("_c11", FloatType(), nullable=True),
    StructField("_c12", FloatType(), nullable=True),
    StructField("_c13", FloatType(), nullable=True),
    StructField("_c14", FloatType(), nullable=True),
    StructField("_c15", FloatType(), nullable=True),
    StructField("_c16", FloatType(), nullable=True),
    StructField("_c17", FloatType(), nullable=True),
    StructField("_c18", FloatType(), nullable=True),
    StructField("_c19", FloatType(), nullable=True),
    StructField("_c21", FloatType(), nullable=True),
    StructField("_c20", FloatType(), nullable=True),
    StructField("_c21", FloatType(), nullable=True),
    StructField("_c22", FloatType(), nullable=True),
    StructField("_c23", FloatType(), nullable=True),
    StructField("_c24", FloatType(), nullable=True),
    StructField("_c25", FloatType(), nullable=True),
    StructField("_c26", FloatType(), nullable=True),
    StructField("_c27", FloatType(), nullable=True),
    StructField("_c28", FloatType(), nullable=True),
    StructField("_c29", FloatType(), nullable=True),
    StructField("_c30", FloatType(), nullable=True),
    StructField("_c31", FloatType(), nullable=True),
    StructField("_c31", FloatType(), nullable=True)]
)

In [ ]:
(tour_df.describe().select(
                    "summary",
                    F.round("a2014_4th_quarter", 4).alias("2014_4Q"),
                    F.round("b2015_4th_quarter", 4).alias("2015_4Q"),
                    F.round("c2016_4th_quarter", 4).alias("2016_4Q"),
                    F.round("d2017_4th_quarter", 4).alias("2017_4Q"),
                    F.round("e2018_4th_quarter", 4).alias("2018_4Q"),
                    F.round("f2019_4th_quarter", 4).alias("2019_4Q"),
                    F.round("g2020_4th_quarter", 4).alias("2020_4Q"),
                    F.round("x", 4).alias("x"),
                    F.round("y", 4).alias("y"))
                    .show())

+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+
|summary|2014_4Q|2015_4Q|2016_4Q|2017_4Q|2018_4Q|2019_4Q|2020_4Q|      x|     y|
+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+
|  count|   62.0|   61.0|   63.0|   58.0|   63.0|   63.0|   63.0|   63.0|  63.0|
|   mean|29.8532|29.2197|29.8222|29.9138|29.0132|29.7968| 7.1905|24.9992|  25.0|
| stddev| 4.8434| 4.6538| 5.1865| 5.5394| 4.9232| 4.6116| 6.5926| 0.0174|0.0149|
|    min|   19.6|   18.8|   18.7|   19.5|   18.7|   21.4|    1.0| 24.975|24.975|
|    max|   44.0|   43.7|   46.6|   49.2|  43.66|   44.8|   36.3| 25.025|25.025|
+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+



### 5.1 Preprocessing The Target Values

In [ ]:
# Adjust the values of `medianHouseValue`
tour_df = tour_df.withColumn("x", col("x")/100000)

In [ ]:
# Show the first 2 lines of `df`
tour_df.show(2)

+----------------------+---+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------+------+-----+
|Country_of_Nationality|CON|a2014_1st_quarter|a2014_2nd_quarter|a2014_3rd_quarter|a2014_4th_quarter|b2015_1st_quarter|b2015_2nd_quarter|b2015_3rd_quarter|b2015_4th_quarter|c2016_1st_quarter|c2016_2nd_quarter|c2016_3rd_quarter|c2016_4th_quarter|d2017_1st_quarter|d2017_2nd_quarter|d2017_3rd_quarter|d2017_4th_quarter|e2018_1st_quarter|e2018_2nd_quarter|e2018_3rd_quarter|e2018_4th_quarter|f2019_1st_quarter|f2019_2nd_quarter|f2019_3rd_quarter|f2019

In [ ]:
tour_df.columns

['Country_of_Nationality',
 'CON',
 'a2014_1st_quarter',
 'a2014_2nd_quarter',
 'a2014_3rd_quarter',
 'a2014_4th_quarter',
 'b2015_1st_quarter',
 'b2015_2nd_quarter',
 'b2015_3rd_quarter',
 'b2015_4th_quarter',
 'c2016_1st_quarter',
 'c2016_2nd_quarter',
 'c2016_3rd_quarter',
 'c2016_4th_quarter',
 'd2017_1st_quarter',
 'd2017_2nd_quarter',
 'd2017_3rd_quarter',
 'd2017_4th_quarter',
 'e2018_1st_quarter',
 'e2018_2nd_quarter',
 'e2018_3rd_quarter',
 'e2018_4th_quarter',
 'f2019_1st_quarter',
 'f2019_2nd_quarter',
 'f2019_3rd_quarter',
 'f2019_4th_quarter',
 'g2020_1st_quarter',
 'g2020_2nd_quarter',
 'g2020_3rd_quarter',
 'g2020_4th_quarter',
 'x',
 'y',
 'T/F']

In [ ]:
# Inspect the result
tour_df.show(5)

+----------------------+---+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+---------+------+-----+
|Country_of_Nationality|CON|a2014_1st_quarter|a2014_2nd_quarter|a2014_3rd_quarter|a2014_4th_quarter|b2015_1st_quarter|b2015_2nd_quarter|b2015_3rd_quarter|b2015_4th_quarter|c2016_1st_quarter|c2016_2nd_quarter|c2016_3rd_quarter|c2016_4th_quarter|d2017_1st_quarter|d2017_2nd_quarter|d2017_3rd_quarter|d2017_4th_quarter|e2018_1st_quarter|e2018_2nd_quarter|e2018_3rd_quarter|e2018_4th_quarter|f2019_1st_quarter|f2019_2nd_quarter|f2019_3rd_quarter|f2

In [ ]:
featureCols = ["x","y"]

In [ ]:
assembler = VectorAssembler(inputCols=featureCols, outputCol="features") 
assembled_df = assembler.transform(tour_df)

In [ ]:
assembled_df.show(10, truncate=False)

+------------------------+---+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+---------+------+-----+------------------+
|Country_of_Nationality  |CON|a2014_1st_quarter|a2014_2nd_quarter|a2014_3rd_quarter|a2014_4th_quarter|b2015_1st_quarter|b2015_2nd_quarter|b2015_3rd_quarter|b2015_4th_quarter|c2016_1st_quarter|c2016_2nd_quarter|c2016_3rd_quarter|c2016_4th_quarter|d2017_1st_quarter|d2017_2nd_quarter|d2017_3rd_quarter|d2017_4th_quarter|e2018_1st_quarter|e2018_2nd_quarter|e2018_3rd_quarter|e2018_4th_quarter|f2019_1st_quarter|f2019_2nd_quart

### 6.2 Standardization

Next, we can finally scale the data using `StandardScaler`. The input columns are the `features`, and the output column with the rescaled that will be included in the scaled_df will be named `"features_scaled"`:

In [ ]:
# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

In [ ]:
# Fit the DataFrame to the scaler
scaled_df = standardScaler.fit(assembled_df).transform(assembled_df)

In [ ]:
# Inspect the result
scaled_df.select("features", "features_scaled").show(10, truncate=False)

+------------------+--------------------------------------+
|features          |features_scaled                       |
+------------------+--------------------------------------+
|[2.5E-4,25.0]     |[1439.1142556911911,1678.744119329124]|
|[2.5E-4,24.975]   |[1439.1142556911911,1677.065375209795]|
|[2.5E-4,25.025]   |[1439.1142556911911,1680.422863448453]|
|[2.4975E-4,25.0]  |[1437.6751414355,1678.744119329124]   |
|[2.4975E-4,24.975]|[1437.6751414355,1677.065375209795]   |
|[2.5E-4,25.0]     |[1439.1142556911911,1678.744119329124]|
|[2.4975E-4,25.025]|[1437.6751414355,1680.422863448453]   |
|[2.5E-4,25.025]   |[1439.1142556911911,1680.422863448453]|
|[2.5E-4,24.975]   |[1439.1142556911911,1677.065375209795]|
|[2.4975E-4,24.975]|[1437.6751414355,1677.065375209795]   |
+------------------+--------------------------------------+
only showing top 10 rows



### 7. Building A Machine Learning Model With Spark ML

With all the preprocessing done, it's finally time to start building our Linear Regression model! Just like always, we first need to split the data into training and test sets. Luckily, this is no issue with the `randomSplit()` method:

In [ ]:
# Split the data into train and test sets
train_data, test_data = scaled_df.randomSplit([.8,.2], seed=rnd_seed)

We pass in a list with two numbers that represent the size that we want your training and test sets to have and a seed, which is needed for reproducibility reasons.

**Note** that the argument `elasticNetParam` corresponds to $\alpha$ or the vertical intercept and that the `regParam` or the regularization paramater corresponds to $\lambda$.

In [ ]:
train_data.columns

['Country_of_Nationality',
 'CON',
 'a2014_1st_quarter',
 'a2014_2nd_quarter',
 'a2014_3rd_quarter',
 'a2014_4th_quarter',
 'b2015_1st_quarter',
 'b2015_2nd_quarter',
 'b2015_3rd_quarter',
 'b2015_4th_quarter',
 'c2016_1st_quarter',
 'c2016_2nd_quarter',
 'c2016_3rd_quarter',
 'c2016_4th_quarter',
 'd2017_1st_quarter',
 'd2017_2nd_quarter',
 'd2017_3rd_quarter',
 'd2017_4th_quarter',
 'e2018_1st_quarter',
 'e2018_2nd_quarter',
 'e2018_3rd_quarter',
 'e2018_4th_quarter',
 'f2019_1st_quarter',
 'f2019_2nd_quarter',
 'f2019_3rd_quarter',
 'f2019_4th_quarter',
 'g2020_1st_quarter',
 'g2020_2nd_quarter',
 'g2020_3rd_quarter',
 'g2020_4th_quarter',
 'x',
 'y',
 'T/F',
 'features',
 'features_scaled']

**Create an ElasticNet model:**

ElasticNet is a linear regression model trained with L1 and L2 prior as regularizer. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge. We control the convex combination of L1 and L2 using the l1_ratio parameter.

Elastic-net is useful when there are multiple features which are correlated with one another. Lasso is likely to pick one of these at random, while elastic-net is likely to pick both.

A practical advantage of trading-off between Lasso and Ridge is it allows Elastic-Net to inherit some of Ridge’s stability under rotation.

The objective function to minimize is in this case:
\begin{align}
min_w\frac{1}{2n_{samples}}{\parallel{X_w - y}\parallel}^2_2 + \alpha\lambda{\parallel{X_w - y}\parallel}_1 + \frac{\alpha(1-\lambda)}{2}{\parallel{w}\parallel}^2_2
\end{align}

http://scikit-learn.org/stable/modules/linear_model.html#elastic-net

In [ ]:
# Initialize `lr`
lr = (LinearRegression(featuresCol='features_scaled', labelCol="x", predictionCol='predx', 
                               maxIter=10, regParam=0.3, elasticNetParam=0.8, standardization=False))

In [ ]:
# Fit the data to the model
linearModel = lr.fit(train_data)

## 8. Evaluating the Model

With our model in place, we can generate predictions for our test data: use the `transform()` method to predict the labels for our `test_data`. Then, we can use RDD operations to extract the predictions as well as the true labels from the DataFrame.

### 8.1 Inspect the Model Co-efficients

In [ ]:
# Coefficients for the model
linearModel.coefficients

DenseVector([0.0, 0.0])

In [ ]:
featureCols

['x', 'y']

In [ ]:
# Intercept for the model
linearModel.intercept

0.0002499850000000001

In [ ]:
coeff_df = pd.DataFrame({"Feature": ["Intercept"] + featureCols, "Co-efficients": np.insert(linearModel.coefficients.toArray(), 0, linearModel.intercept)})
coeff_df = coeff_df[["Feature", "Co-efficients"]]

In [ ]:
coeff_df

,Feature,Co-efficients
0,Intercept,0.00025
1,x,0.00000
2,y,0.00000


### 8.2 Generating Predictions

In [ ]:
# Generate predictions
predictions = linearModel.transform(test_data)

In [ ]:
# Extract the predictions and the "known" correct labels
predandlabels = predictions.select("predx", "x")

In [ ]:
predandlabels.show()

+--------------------+---------+
|               predx|        x|
+--------------------+---------+
|2.499850000000001E-4|   2.5E-4|
|2.499850000000001E-4|2.5025E-4|
|2.499850000000001E-4|2.4975E-4|
|2.499850000000001E-4|   2.5E-4|
|2.499850000000001E-4|   2.5E-4|
|2.499850000000001E-4|   2.5E-4|
|2.499850000000001E-4|2.4975E-4|
|2.499850000000001E-4|   2.5E-4|
|2.499850000000001E-4|   2.5E-4|
|2.499850000000001E-4|2.5025E-4|
|2.499850000000001E-4|   2.5E-4|
|2.499850000000001E-4|   2.5E-4|
|2.499850000000001E-4|2.5025E-4|
+--------------------+---------+



### 8.3 Inspect the Metrics

Looking at predicted values is one thing, but another and better thing is looking at some metrics to get a better idea of how good your model actually is.

**Using the `LinearRegressionModel.summary` attribute:**

Next, we can also use the `summary` attribute to pull up the `rootMeanSquaredError` and the `r2`.

In [ ]:
# Get the RMSE
print("RMSE: {0}".format(linearModel.summary.rootMeanSquaredError))

RMSE: 1.7613914953807058e-07


In [ ]:
print("MAE: {0}".format(linearModel.summary.meanAbsoluteError))

MAE: 1.3159999999995028e-07


In [ ]:
# Get the R2
print("R2: {0}".format(linearModel.summary.r2))

R2: -4.463096558993129e-14


+ The RMSE measures how much error there is between two datasets comparing a predicted value and an observed or known value. The smaller an RMSE value, the closer predicted and observed values are.

+ The R2 ("R squared") or the coefficient of determination is a measure that shows how close the data are to the fitted regression line. This score will always be between 0 and a 100% (or 0 to 1 in this case), where 0% indicates that the model explains none of the variability of the response data around its mean, and 100% indicates the opposite: it explains all the variability. That means that, in general, the higher the R-squared, the better the model fits our data.

**Using the RegressionEvaluator from pyspark.ml package:**

In [ ]:
evaluator = RegressionEvaluator(predictionCol="predx", labelCol='x', metricName='rmse')
print("RMSE: {0}".format(evaluator.evaluate(predandlabels)))

RMSE: 1.576083266054634e-07


In [ ]:
evaluator = RegressionEvaluator(predictionCol="predx", labelCol='x', metricName='mae')
print("MAE: {0}".format(evaluator.evaluate(predandlabels)))

MAE: 1.0653846153839177e-07


In [ ]:
evaluator = RegressionEvaluator(predictionCol="predx", labelCol='x', metricName='r2')
print("R2: {0}".format(evaluator.evaluate(predandlabels)))

R2: -0.04950624999957909
